# Import Packages

In [1]:
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from tensorflow.keras import backend as K
import tensorflow as tf
import tarfile

from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

from io import BytesIO
from PIL import Image
import gzip
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import data_handler as dh
import utility_handler as uh
import backbond_handler as bh

r_seed = 0

# To load known-unknown class setting as the same as
# The details can be found here:
# https://github.com/iCGY96/ARPL/blob/master/split.py
num_shuffle = len(uh.splits_2020['cifar10'])
data_load_func = dh.load_cifar10

backbone = bh.VGG1132
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epoch_max = 120

# Loading Data

In [2]:
trX, trY, teX, teY = data_load_func()

# Encoding

In [3]:
num_class = 10
label_set = np.arange(num_class)
kn_list = uh.splits_2020['cifar10']
uk_list = []

for kn_labels in kn_list:
    temp_uk_label_list = np.setdiff1d(label_set, kn_labels)
    uk_list.append(temp_uk_label_list.tolist())

In [4]:
# ====================================================== #
# random split 10 classes into 6 known 4 unknown classes #
# ====================================================== #
dataset_X_dict, dataset_Y_dict = uh.split_know_unkn_mnist_svhn_cifar10(
    trX,
    trY,
    teX,
    teY,
    num_shuffle=num_shuffle,
    know_class_list=kn_list,
    unkn_class_list=uk_list,
    r_seed=r_seed)

# ============= #
# build encoder #
# ============= #
encoder_list = bh.build_encoder(dataset_X_dict, dataset_Y_dict, backbone,
                                device, r_seed, epoch_max,
                                data_load_func.__name__)

# ============= #
# encode images #
# ============= #
kn_trX_encode_list = []
kn_teX_encode_list = []
uk_teX_encode_list = []

# shuffle is enabled during traing
# therefore, we need to store the label at encoding stage
kn_trY_encode_list = []
kn_teY_encode_list = []
uk_teY_encode_list = []

for idx, encoder in enumerate(encoder_list):
    kn_tr_Dataset = bh.image_data_to_dataloader(dataset_X_dict['kn_tr'][idx],
                                                dataset_Y_dict['kn_tr'][idx],
                                                128)
    kn_te_Dataset = bh.image_data_to_dataloader(dataset_X_dict['kn_te'][idx],
                                                dataset_Y_dict['kn_te'][idx],
                                                128)
    uk_te_Dataset = bh.image_data_to_dataloader(dataset_X_dict['uk_te'][idx],
                                                dataset_Y_dict['uk_te'][idx],
                                                128)

    kn_tr_encode_XY = bh.encoding(kn_tr_Dataset, encoder, device)
    kn_te_encode_XY = bh.encoding(kn_te_Dataset, encoder, device)
    uk_te_encode_XY = bh.encoding(uk_te_Dataset, encoder, device)

    kn_trX_encode_list.append(kn_tr_encode_XY[:, :-1])
    kn_teX_encode_list.append(kn_te_encode_XY[:, :-1])
    uk_teX_encode_list.append(uk_te_encode_XY[:, :-1])

    kn_trY_encode_list.append(kn_tr_encode_XY[:, -1])
    kn_teY_encode_list.append(kn_te_encode_XY[:, -1])
    uk_teY_encode_list.append(uk_te_encode_XY[:, -1])

numpy random seed 0
Ranodm set 0
    know classes [0, 6, 4, 9, 1, 7]
    unkn classes [2, 3, 5, 8]
Ranodm set 1
    know classes [7, 6, 4, 9, 0, 1]
    unkn classes [2, 3, 5, 8]
Ranodm set 2
    know classes [1, 5, 7, 3, 9, 4]
    unkn classes [0, 2, 6, 8]
Ranodm set 3
    know classes [8, 6, 1, 9, 0, 7]
    unkn classes [2, 3, 4, 5]
Ranodm set 4
    know classes [2, 4, 1, 7, 9, 6]
    unkn classes [0, 3, 5, 8]
channel 3
93.86666666666666 tensor(0.1765, device='cuda:0', grad_fn=<NllLossBackward>)
93.86666666666666 tensor(0.2453, device='cuda:0', grad_fn=<NllLossBackward>)
88.55 tensor(0.3265, device='cuda:0', grad_fn=<NllLossBackward>)
92.66666666666667 tensor(0.1950, device='cuda:0', grad_fn=<NllLossBackward>)
92.33333333333333 tensor(0.4411, device='cuda:0', grad_fn=<NllLossBackward>)


# Open-set Enrichment

In [5]:
from aosr_utility import sample_enrichment_IF

kn_trX_encode_zqX_list = []
kn_trX_encode_zqW_list = []

for i in range(num_shuffle):
    encX = kn_trX_encode_list[i]
    zqX, zqW = sample_enrichment_IF(0, encX, encX.shape[0] * 3)
    zqW = 1 - zqW * 1.0

    tau = np.sort(zqW)[int(zqW.shape[0] * .001)]

    # Unknown Samples
    cond1 = (zqW > tau)
    cond1 = np.squeeze(cond1)
    # Known Samples
    cond2 = (zqW <= tau)
    cond2 = np.squeeze(cond2)
    # Wipe Known Samples
    zqW[cond2] = 0
    # Balance the weights
    zqW = zqW * (zqW.shape[0] / zqW.sum())

    kn_trX_encode_zqX_list.append(zqX)
    kn_trX_encode_zqW_list.append(zqW)

(56252,)
(55469,)
(64527,)
(53632,)
(62034,)


# Open-set Learning

In [6]:
from aosr_utility import aosr_risk, EarlyStoppingBeforeOverfit
from sklearn.preprocessing import LabelEncoder

for i in range(num_shuffle):
    le = LabelEncoder()
    kn_trY_encode_list[i] = le.fit_transform(kn_trY_encode_list[i])
    kn_teY_encode_list[i] = le.transform(kn_teY_encode_list[i])


def build_openset_learner(zpX, zpY, zqX, zqW, mu, num_class):

    bh.set_random_seed(r_seed, device)

    detector = tf.keras.models.Sequential([
        tf.keras.layers.Dense(num_class + 1),
        tf.keras.layers.Activation(activation='softmax')
    ])

    detector.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
    detector.fit(zpX, zpY * 1.0, epochs=2)

    detector.compile(optimizer='adam',
                     loss=aosr_risk(detector, zqX, zqW, zpX, mu, num_class),
                     metrics=['accuracy'])

    K.set_value(detector.optimizer.learning_rate, 0.001)
    detector.fit(zpX,
                 zpY,
                 epochs=100,
                 callbacks=[EarlyStoppingBeforeOverfit()])
    return detector


mu = 0.05
num_class = np.unique(kn_trY_encode_list[0]).shape[0]
print(mu, num_class)
detector_list = []

for i in range(num_shuffle):

    zpX = kn_trX_encode_list[i]
    zpY = kn_trY_encode_list[i]
    zqX = kn_trX_encode_zqX_list[i]
    zqW = kn_trX_encode_zqW_list[i]
    detector = build_openset_learner(zpX, zpY, zqX, zqW, mu, num_class)
    detector_list.append(detector)

0.05 6
Epoch 1/2
938/938 [==============================] - 2s 1ms/step - loss: 4.9230 - accuracy: 0.2816
Epoch 2/2
938/938 [==============================] - 1s 1ms/step - loss: 0.0687 - accuracy: 0.9946
Epoch 1/100
938/938 [==============================] - 3s 3ms/step - loss: 1152715100.4899 - accuracy: 0.9977
Epoch 2/100
938/938 [==============================] - 2s 3ms/step - loss: 7053.8399 - accuracy: 0.9978
Epoch 3/100
938/938 [==============================] - 2s 3ms/step - loss: 7049.7288 - accuracy: 0.9980
Epoch 4/100
938/938 [==============================] - 2s 3ms/step - loss: 7042.5904 - accuracy: 0.9979
Epoch 5/100
938/938 [==============================] - 2s 2ms/step - loss: 7031.2421 - accuracy: 0.9978
Epoch 6/100
938/938 [==============================] - 2s 2ms/step - loss: 7012.8641 - accuracy: 0.9979
Epoch 7/100
938/938 [==============================] - 1s 2ms/step - loss: 6983.4751 - accuracy: 0.9973
Epoch 8/100
938/938 [==============================] - 1s 2ms

# Evaluation

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import f1_score

In [8]:
actY_list = []
preY_list = []

actY_know_list = []
preY_know_list = []

actY_binary_list = []
preY_binary_list = []

for i in range(num_shuffle):
    
    actY = np.zeros(kn_teX_encode_list[i].shape[0] + uk_teX_encode_list[i].shape[0])
    actY[:kn_teX_encode_list[i].shape[0]] = kn_teY_encode_list[i]
    actY[kn_teX_encode_list[i].shape[0]:] = num_class
    
    kn_preY = detector_list[i].predict(kn_teX_encode_list[i])
    uk_preY = detector_list[i].predict(uk_teX_encode_list[i])
    preY = np.concatenate([kn_preY, uk_preY], axis=0)
    
    actY_know_list.append(kn_teY_encode_list[i])
    preY_know_list.append(kn_preY.argmax(axis=1))
    
    actY_list.append(actY)
    preY_list.append(preY)
    
    actY_bi = np.array(actY)
    preY_bi = preY[:, -1]
    
    actY_bi[actY_bi<6] = 0
    actY_bi[actY_bi==6] = 1
    
    actY_binary_list.append(actY_bi)
    preY_binary_list.append(preY_bi)

In [9]:
acc_list = []
auc_bi_list = []
auc_list = []
f1_list = []
for i in range(num_shuffle):
    print(f'shuffle {i}')
    print('    acc score', accuracy_score(actY_know_list[i], preY_know_list[i]))
    print('    acc known', accuracy_score(actY_know_list[i][actY_know_list[i]==0], preY_know_list[i][actY_know_list[i]==0]))
    print('    acc uknow', accuracy_score(actY_know_list[i][actY_know_list[i]==1], preY_know_list[i][actY_know_list[i]==1]))
    acc_list.append(roc_auc_score(actY_list[i], preY_list[i], multi_class='ovr', average='weighted'))
    auc_bi_list.append(roc_auc_score(actY_binary_list[i], preY_binary_list[i]))
    fpr, tpr, thresholds = metrics.roc_curve(actY_binary_list[i], preY_binary_list[i], pos_label=1)
    auc_list.append(metrics.auc(fpr, tpr))
    f1_list.append(f1_score(actY_list[i], preY_list[i].argmax(1), average='macro'))
    print('    f1 macro score', f1_list[i])
    print('    auc only score', auc_list[i])
    print('    auc multi-score', acc_list[i])
    print('    auc binary score', auc_bi_list[i])
print(f'Avg F1 {np.mean(f1_list)}, {np.std(f1_list)}')
print(f'Avg AUC {np.mean(auc_list)}, {np.std(auc_list)}')
print(f'Avg Accurac {np.mean(acc_list)}, {np.std(acc_list)}')
print(f'Avg ROC_AUC {np.mean(auc_bi_list)}, {np.std(auc_bi_list)}')

shuffle 0
    acc score 0.8475
    acc known 0.846
    acc uknow 0.876
    f1 macro score 0.7709703069722599
shuffle 1
    acc score 0.9368333333333333
    acc known 0.947
    acc uknow 0.938
    f1 macro score 0.6196618790430819
shuffle 2
    acc score 0.778
    acc known 0.918
    acc uknow 0.59
    f1 macro score 0.6975415335736325
shuffle 3
    acc score 0.8705
    acc known 0.867
    acc uknow 0.949
    f1 macro score 0.7403032682558877
shuffle 4
    acc score 0.8538333333333333
    acc known 0.961
    acc uknow 0.874
    f1 macro score 0.6984611291947489
Avg F1 0.7053876234079223, 0.050948270497295826
Avg Accurac 0.898251701111111, 0.02473986267194473
